In [1]:
import nibabel as nib
import numpy as np
import os
import pickle as pkl

In [2]:
import os
import torch
os.environ['CUDA_VISIBLE_DEVICES']='2'

In [ ]:
# Looking at information from checkpoint of new 5 sample data

In [3]:
modalities = ['_t1.nii.gz','_t2.nii.gz', '_t1ce.nii.gz', '_flair.nii.gz', '_seg.nii.gz']

In [4]:
sample_pardir = '/raid/datasets/BraTS22/BraTS2022_Training/RSNA_ASNR_MICCAI_BraTS2021_TrainingData_16July2021'
patients = ['BraTS2021_00512', 'BraTS2021_00565', 'BraTS2021_01200', 'BraTS2021_01277', 'BraTS2021_01394']

In [5]:
dir_path_tups = [(os.path.join(sample_pardir, patient), patient) for patient in patients]

In [6]:
seen_shapes = []
for dir_path, patient in dir_path_tups:
    for mod in modalities:
        path = os.path.join(dir_path,patient + mod)
        shape = np.array(nib.load(path).dataobj).astype(np.float32).shape
        if shape not in seen_shapes:
            seen_shapes.append(shape)
seen_shapes



[(240, 240, 155)]

In [7]:
# What about the cropped data?

cropped_543_pardir = '/raid/edwardsb/projects/RANO/BraTS22_pretending_tobe_postopp/nnUNet_raw_data_base/nnUNet_cropped_data/Task543_FakePostOpp_More/'
cropped_542_pardir = '/raid/edwardsb/projects/RANO/BraTS22_pretending_tobe_postopp/nnUNet_raw_data_base/nnUNet_cropped_data/Task542_FakePostOpp/'

In [14]:
def inspect_shapes(cropped_pardir):
    seen_shapes = []
    for _file in os.listdir(cropped_pardir):
        if _file.endswith('.pkl') or _file.endswith('.json') or not os.path.isfile(os.path.join(cropped_pardir, _file)):
            # print(f"skipped file: {_file}")
            continue
        # print(_file)
        path = os.path.join(cropped_pardir, _file)
        array = np.load(path)['data']
        shape = array.shape
        # print(shape)
        if shape not in seen_shapes:
            seen_shapes.append(shape)
    print(f"unique cropped shapes were: {seen_shapes}")

In [15]:
inspect_shapes(cropped_543_pardir)

unique cropped shapes were: [(5, 137, 173, 128), (5, 130, 159, 123), (5, 136, 171, 130), (5, 141, 176, 141), (5, 138, 168, 135), (5, 144, 169, 133)]


In [16]:
inspect_shapes(cropped_542_pardir)

unique cropped shapes were: [(5, 133, 160, 136)]


In [10]:
# inspect checkpoints to see what patch sized ect were selected for each task
checkpoint_542_path = '/raid/edwardsb/projects/RANO/NNUnetModels/nnUNet/3d_fullres/Task542_FakePostOpp/nnUNetTrainerV2__nnUNetPlansv2.1/all/model_final_checkpoint.model'
checkpoint_543_path = '/raid/edwardsb/projects/RANO/NNUnetModels/nnUNet/3d_fullres/Task543_FakePostOpp_More/nnUNetTrainerV2__nnUNetPlansv2.1/fold_0/model_final_checkpoint.model'

In [11]:
import torch
ch_42 = torch.load(checkpoint_542_path)
ch_43 = torch.load(checkpoint_543_path)

In [12]:
ch_42.keys()

dict_keys(['epoch', 'state_dict', 'optimizer_state_dict', 'lr_scheduler_state_dict', 'plot_stuff', 'best_stuff', 'amp_grad_scaler'])

In [13]:
ch_42['state_dict'].keys()

odict_keys(['conv_blocks_localization.0.0.blocks.0.conv.weight', 'conv_blocks_localization.0.0.blocks.0.conv.bias', 'conv_blocks_localization.0.0.blocks.0.instnorm.weight', 'conv_blocks_localization.0.0.blocks.0.instnorm.bias', 'conv_blocks_localization.0.1.blocks.0.conv.weight', 'conv_blocks_localization.0.1.blocks.0.conv.bias', 'conv_blocks_localization.0.1.blocks.0.instnorm.weight', 'conv_blocks_localization.0.1.blocks.0.instnorm.bias', 'conv_blocks_localization.1.0.blocks.0.conv.weight', 'conv_blocks_localization.1.0.blocks.0.conv.bias', 'conv_blocks_localization.1.0.blocks.0.instnorm.weight', 'conv_blocks_localization.1.0.blocks.0.instnorm.bias', 'conv_blocks_localization.1.1.blocks.0.conv.weight', 'conv_blocks_localization.1.1.blocks.0.conv.bias', 'conv_blocks_localization.1.1.blocks.0.instnorm.weight', 'conv_blocks_localization.1.1.blocks.0.instnorm.bias', 'conv_blocks_localization.2.0.blocks.0.conv.weight', 'conv_blocks_localization.2.0.blocks.0.conv.bias', 'conv_blocks_localiz

In [11]:
# Where is the patch size defined?
plans_path_42 = '/raid/edwardsb/projects/RANO/NNUnetModels/nnUNet/3d_fullres/Task542_FakePostOpp/nnUNetTrainerV2__nnUNetPlansv2.1/plans.pkl'
plans_path_43 = '/raid/edwardsb/projects/RANO/NNUnetModels/nnUNet/3d_fullres/Task543_FakePostOpp_More/nnUNetTrainerV2__nnUNetPlansv2.1/plans.pkl'

In [2]:
def load_pickle(path):
    with open(path, 'rb') as _file:
        return pkl.load(_file)

In [13]:
plans_42 = load_pickle(plans_path_42)
plans_43 = load_pickle(plans_path_43)


In [19]:
plan_keys = plans_42.keys()
plan_keys

dict_keys(['num_stages', 'num_modalities', 'modalities', 'normalization_schemes', 'dataset_properties', 'list_of_npz_files', 'original_spacings', 'original_sizes', 'preprocessed_data_folder', 'num_classes', 'all_classes', 'base_num_features', 'use_mask_for_norm', 'keep_only_largest_region', 'min_region_size_per_class', 'min_size_per_class', 'transpose_forward', 'transpose_backward', 'data_identifier', 'plans_per_stage', 'preprocessor_name', 'conv_per_stage'])

In [34]:
diff_dict = {}
for key in plan_keys:
    different = False
    if isinstance(plans_42[key], np.ndarray):
        if not np.all(plans_42[key] == plans_43[key]):
            different = True
    elif isinstance(plans_42[key], int) and not isinstance(plans_42[key], list):
        different = plans_42[key] != plans_43[key]
    elif isinstance(plans_42[key], dict):
        print(f"{key}: Skipping DICT comparison!!")
        continue
    else:
        print(f"{key}: Skipping comparison of value with type: {type(plans_42[key])}")
        continue

    if different:
        diff_dict[key] = {'42': plans_42[key], '43': plans_43[key]}
diff_dict

modalities: Skipping DICT comparison!!
normalization_schemes: Skipping DICT comparison!!
dataset_properties: Skipping DICT comparison!!
list_of_npz_files: Skipping comparison of value with type: <class 'list'>
original_spacings: Skipping comparison of value with type: <class 'list'>
original_sizes: Skipping comparison of value with type: <class 'list'>
preprocessed_data_folder: Skipping comparison of value with type: <class 'str'>
all_classes: Skipping comparison of value with type: <class 'list'>
use_mask_for_norm: Skipping DICT comparison!!
keep_only_largest_region: Skipping comparison of value with type: <class 'NoneType'>
min_region_size_per_class: Skipping comparison of value with type: <class 'NoneType'>
min_size_per_class: Skipping comparison of value with type: <class 'NoneType'>
transpose_forward: Skipping comparison of value with type: <class 'list'>
transpose_backward: Skipping comparison of value with type: <class 'list'>
data_identifier: Skipping comparison of value with t

{}

In [35]:
plans_42['dataset_properties']

{'all_sizes': [(133, 160, 136),
  (133, 160, 136),
  (133, 160, 136),
  (133, 160, 136),
  (133, 160, 136),
  (133, 160, 136)],
 'all_spacings': [array([1., 1., 1.]),
  array([1., 1., 1.]),
  array([1., 1., 1.]),
  array([1., 1., 1.]),
  array([1., 1., 1.]),
  array([1., 1., 1.])],
 'all_classes': [1, 2, 3, 4],
 'modalities': {0: '0000', 1: '0001', 2: '0002', 3: '0003'},
 'intensityproperties': None,
 'size_reductions': OrderedDict([('BraTS2021_01030', 0.32415770609318995),
              ('BraTS2021_51030', 0.32415770609318995),
              ('BraTS2021_61030', 0.32415770609318995),
              ('BraTS2021_71030', 0.32415770609318995),
              ('BraTS2021_81030', 0.32415770609318995),
              ('BraTS2021_91030', 0.32415770609318995)])}

In [36]:
plans_43['dataset_properties']

{'all_sizes': [(144, 169, 133),
  (136, 171, 130),
  (137, 173, 128),
  (130, 159, 123),
  (141, 176, 141),
  (138, 168, 135)],
 'all_spacings': [array([1., 1., 1.]),
  array([1., 1., 1.]),
  array([1., 1., 1.]),
  array([1., 1., 1.]),
  array([1., 1., 1.]),
  array([1., 1., 1.])],
 'all_classes': [1, 2, 3, 4],
 'modalities': {0: '0000', 1: '0001', 2: '0002', 3: '0003'},
 'intensityproperties': None,
 'size_reductions': OrderedDict([('BraTS2021_00468', 0.3625322580645161),
              ('BraTS2021_00512', 0.3386290322580645),
              ('BraTS2021_00565', 0.3397992831541219),
              ('BraTS2021_01200', 0.28476814516129034),
              ('BraTS2021_01277', 0.39191935483870965),
              ('BraTS2021_01394', 0.3505645161290323)])}

In [37]:
plans_42['original_sizes']

[(133, 160, 136),
 (133, 160, 136),
 (133, 160, 136),
 (133, 160, 136),
 (133, 160, 136),
 (133, 160, 136)]

In [38]:
plans_43['original_sizes']

[(144, 169, 133),
 (136, 171, 130),
 (137, 173, 128),
 (130, 159, 123),
 (141, 176, 141),
 (138, 168, 135)]

In [39]:
plans_43['plans_per_stage']

{0: {'batch_size': 2,
  'num_pool_per_axis': [5, 5, 4],
  'patch_size': array([128, 160, 112]),
  'median_patient_size_in_voxels': array([138, 170, 132]),
  'current_spacing': array([1., 1., 1.]),
  'original_spacing': array([1., 1., 1.]),
  'do_dummy_2D_data_aug': False,
  'pool_op_kernel_sizes': [[2, 2, 2],
   [2, 2, 2],
   [2, 2, 2],
   [2, 2, 2],
   [2, 2, 1]],
  'conv_kernel_sizes': [[3, 3, 3],
   [3, 3, 3],
   [3, 3, 3],
   [3, 3, 3],
   [3, 3, 3],
   [3, 3, 3]]}}

In [40]:
plans_42['plans_per_stage']

{0: {'batch_size': 2,
  'num_pool_per_axis': [5, 5, 5],
  'patch_size': array([128, 128, 128]),
  'median_patient_size_in_voxels': array([133, 160, 136]),
  'current_spacing': array([1., 1., 1.]),
  'original_spacing': array([1., 1., 1.]),
  'do_dummy_2D_data_aug': False,
  'pool_op_kernel_sizes': [[2, 2, 2],
   [2, 2, 2],
   [2, 2, 2],
   [2, 2, 2],
   [2, 2, 2]],
  'conv_kernel_sizes': [[3, 3, 3],
   [3, 3, 3],
   [3, 3, 3],
   [3, 3, 3],
   [3, 3, 3],
   [3, 3, 3]]}}

In [4]:
splits_info = load_pickle('/raid/edwardsb/projects/RANO/BraTS22_pretending_tobe_postopp/nnUNet_raw_data_base/nnUNet_preprocessed/Task543_FakePostOpp_More/splits_final.pkl')

In [6]:
splits_info

[OrderedDict([('train',
               array(['BraTS2021_00468', 'BraTS2021_00512', 'BraTS2021_00565',
                      'BraTS2021_01277'], dtype='<U15')),
              ('val',
               array(['BraTS2021_01200', 'BraTS2021_01394'], dtype='<U15'))]),
 OrderedDict([('train',
               array(['BraTS2021_00468', 'BraTS2021_00512', 'BraTS2021_00565',
                      'BraTS2021_01200', 'BraTS2021_01394'], dtype='<U15')),
              ('val', array(['BraTS2021_01277'], dtype='<U15'))]),
 OrderedDict([('train',
               array(['BraTS2021_00512', 'BraTS2021_00565', 'BraTS2021_01200',
                      'BraTS2021_01277', 'BraTS2021_01394'], dtype='<U15')),
              ('val', array(['BraTS2021_00468'], dtype='<U15'))]),
 OrderedDict([('train',
               array(['BraTS2021_00468', 'BraTS2021_00565', 'BraTS2021_01200',
                      'BraTS2021_01277', 'BraTS2021_01394'], dtype='<U15')),
              ('val', array(['BraTS2021_00512'], dtype='<U15'))]

In [ ]:
#################################
#################################
################################

In [2]:
import os

"""
Creating some data links that mimic the folder structure expected after data prep

"""
source_dirpath = '/raid/datasets/BraTS22/BraTS2022_Training/RSNA_ASNR_MICCAI_BraTS2021_TrainingData_16July2021'
# the directory immediately below should be cleared of all items before running this again
test_folderpath = '/raid/edwardsb/projects/RANO/test_data_links'

src_mods = ['t1.nii.gz', 't2.nii.gz', 't1ce.nii.gz', 'flair.nii.gz', 'seg.nii.gz']
dst_mods = ['t1n.nii.gz', 't2w.nii.gz', 't1c.nii.gz', 't2f.nii.gz', 'final_seg.nii.gz']

patients = list(os.listdir(source_dirpath))[:10]
for src_mod, dst_mod in zip(src_mods, dst_mods):
    if src_mod == 'seg.nii.gz':
        subdir = 'labels'
    else:
        subdir = 'data'
    
    for patient in patients:
        dst_pat_dir = os.path.join(test_folderpath, subdir, patient)
        if not os.path.exists(dst_pat_dir):
            os.makedirs(dst_pat_dir)
        src_pat_dir = os.path.join(source_dirpath, patient)
        for timestamp in ['2008.03.88', '2008.12.99']:
            dst_pattim_dir = os.path.join(dst_pat_dir, timestamp)
            if not os.path.exists(dst_pattim_dir):
                os.makedirs(dst_pattim_dir)
       
            


            src_path = os.path.join(src_pat_dir, patient + '_' + src_mod)
            if dst_mod == 'final_seg.nii.gz':
                dst_path = os.path.join(dst_pattim_dir, patient + '_' + timestamp + '_' + dst_mod)
            else:
                dst_path = os.path.join(dst_pattim_dir, patient + '_' + timestamp + '_brain_' + dst_mod)
            os.symlink(src=src_path, dst=dst_path)
        



In [21]:
# Ok, I'm getting some errors in NNUnet data preprocessing with spacing info not being there. I had

import SimpleITK as sitk
import shutil

def BE_version_split_4d_nifti(filename, output_folder, add_zeros=False):
    img_itk = sitk.ReadImage(filename)
    dim = img_itk.GetDimension()
    print(f"Dimension is: {dim}")
    file_base = filename.split("/")[-1]
    img_npy = sitk.GetArrayFromImage(img_itk)
    spacing = img_itk.GetSpacing()
    origin = img_itk.GetOrigin()
    print("###########################")
    print(spacing)
    print("###########################")
    # Brandon stopping here to output for test

In [22]:
BE_version_split_4d_nifti('/raid/edwardsb/projects/RANO/BraTS22_pretending_tobe_postopp/nnUNet_raw_data_base/nnUNet_raw_data/Task542_FakePostOpp/imagesTr/BraTS2021_01030_0000.nii.gz', './delete_me_test')

Dimension is: 3
###########################
(1.0, 1.0, 1.0)
###########################


In [29]:
BE_version_split_4d_nifti('/raid/edwardsb/projects/RANO/BraTS22_pretending_tobe_postopp/nnUNet_raw_data_base/nnUNet_raw_data/Task544_PostOpp_PreTest/imagesTr/BraTS2021_01630_2008.12.99_0000.nii.gz', './delete_me_test')

RuntimeError: Exception thrown in SimpleITK ImageFileReader_Execute: /tmp/SimpleITK/Code/IO/src/sitkImageReaderBase.cxx:97:
sitk::ERROR: The file "/raid/edwardsb/projects/RANO/BraTS22_pretending_tobe_postopp/nnUNet_raw_data_base/nnUNet_raw_data/Task544_PostOpp_PreTest/imagesTr/BraTS2021_01630_2008.12.99_0000.nii.gz" does not exist.

In [6]:
[1, 2, 3, 4, 5, 6, 7, 8, 9][::2], [1, 2, 3, 4, 5, 6, 7, 8, 9][1::2]

([1, 3, 5, 7, 9], [2, 4, 6, 8])